# Team15_Project1

This is the deliverable ipynb file for BIOINF 575 Bioinformatics project assignment.

Team 15 members: John Larson, Henry Li, Ziyan Xu

Project topic: Drug Annotation of 23andme Report

## Input Data

23andme_v5_hg19_ref.txt.gz - retrieved at [23andme2vcf](https://github.com/arrogantrobot/23andme2vcf)

var_drug_ann.tsv - retrieved at [ClinPGx](https://www.clinpgx.org/downloads) (previously PharmGKB) as variantAnnotations.zip

the file was extracted and brought to the repository root directory for ease of access

## Task 1

**Map/merge** the 23andme file and the variant-drug annotation file based on dbSNP_ID (aka rsID).

The merged result should have the following columns:

```
dbSNP_ID, GENE_SYMBOL, DRUG_NAME, PMID, PHENOTYPE_CATEGORY, SIGNIFICANCE, NOTES, SENTENCE, ALLELE_PharmGKB (variant alleles in annotation), ALLELE_23andme (variant alleles in 23andme file)
```

## Task 2

**Filter** the output so that it only contains significant associations (**SIGNIFICANCE** is **yes**) for variants that affect the drug efficacy (**PHENOTYPE_CATEGORY** is **efficacy**).

## Task 3

**Save** the output of the filtering step in a tab-separated file (**23andme_PharmGKB_map.tsv**) with the following columns:

```
dbSNP_ID, GENE_SYMBOL, DRUG_NAME, NOTES, SENTENCE, ALLELE_PharmGKB, ALLELE_23andme
```

## Task 4

**Create** a tab separated file (**23andme_PharmGKB_summary.tsv**) with summarized data with the following columns:

```
GENE_SYMBOL, DRUG_NAME, dbSNP_IDs (list of IDs separated by ";")
```

## Task 5

**Plot** the distribution (histogram) of the number of drugs associated with a gene, and the number of SNPs for a gene.

## Task 6

As a team decide on a new feature for each of the team members to implement. These new features should be included in the solution and should answer a relevant biological question using the data available for the project. New data can be used for the implementation of these new features.

In [ ]:
# John's New Feature

In [ ]:
# Henry's New Feature

In [ ]:
# Ziyan's New Feature